# Counts per researcher

By **Franklin Oliveira**

-----
This notebook contains all code necessary to make the "type" charts from `poliqueta` database. Here you'll find some basic data treatment and charts' code. 

Database: <font color='blue'>'IBUFRJ27.07.2020 - visualização.xlsx'</font>.

In [1]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# quick visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Waffle Charts
# from pywaffle import Waffle 
# docs: https://pywaffle.readthedocs.io/en/latest/examples/block_shape_distance_location_and_direction.html

# visualization
import altair as alt

# enabling notebook renderer
# alt.renderers.enable('notebook')
alt.renderers.enable('default')

# disabling rows limit
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

In [2]:
NewTable = pd.read_csv('./data/merged_db.csv', sep=';', encoding='utf-8', low_memory=False)

<br>

<font size=5>**Color Palette per Order**</font>

These images were used as inspiration (https://color.adobe.com/create/image)


<div class='row' style='padding-top:20px;'>
    <div class='col-md-6'>
        <img src="./src/img1.jpg" width='400px'>
    </div>
    <div class='col-md-6'>
        <img src="./src/img2.jpg" width='400px'>
    </div>
</div>

<br>

A partir das imagens acima, selecionamos cores (centróides) para criar a paleta de cores. Foram elas: 
<ul>
    <li style='color:#3CA67F'><b> #3CA67F </b># verde</li>
    <li style='color:#7A9FBF'><b> #7A9FBF </b># azul</li>
    <li style='color:#D94814'><b> #D94814 </b># laranja</li>
    <li style='color:#D96236'><b> #D96236 </b># laranja 2</li>
    <li style='color:#F2B999'><b> #F2B999 </b># 'cor de pele'</li>
    <li style='color:#A66C4B'><b> #A66C4B </b># marrom 1</li>
    <li style='color:#732C02'><b> #732C02 </b># marrom 2</li>
</ul>

A partir das cores "centróides", utilizamos a ferramenta Color Crafter para selecionar diferentes "shades" e auxiliar para categorização em diferentes grupos sugeridos pela equipe de Poliquetas do Museu Nacional. 

<ul>
    <li style='color:#3CA67F'><b> Verde: </b> ['#daffef', '#bbebd3', '#9adabc', '#77c8a5', '#57b791', '#3ca67f', '#2a9670', '#238762', '#257a56']</li>
    <li style='color:#7A9FBF'><b> Azul: </b> ['#e7e5df', '#ccd2d8', '#b2c0d0', '#96afc8', '#7a9fbf', '#5d90b6', '#3c81ae', '#0673a4', '#00669a']</li>
    <li style='color:#D94814'><b> laranja: </b> ['#ffbd84', '#ffaa74', '#ff9760', '#ff814b', '#fc6b36', '#eb5824', '#d94814', '#c83b03', '#b73000']</li>
    <li style='color:#D96236'><b> laranja 2: ['#ffeba9', '#ffd391', '#ffbb7b', '#fda468', '#f18e56', '#e57846', '#d96236', '#cc4d28', '#bf381b']</b> </li>
    <li style='color:#F2B999'><b> cor de pele: ['#ffe9c3', '#fbd0ad', '#f2b999', '#e8a287', '#dd8c76', '#d27666', '#c76158', '#bb4d4b', '#ae393e']</b> </li>
    <li style='color:#A66C4B'><b> marrom 1: ['#d9c6af', '#ccad96', '#c1977c', '#b48061', '#a66c4b', '#975b39', '#874c2c', '#774124', '#683720']</b> </li>
    <li style='color:#732C02'><b> marrom 2: ['#eebd93', '#dfa47a', '#d28d60', '#c37746', '#b4622f', '#a3501d', '#92420e', '#823606', '#732c02']</b> </li>
</ul>



**Colors  (antigas):** 

<ul>
    <li style='color:#41A681'><b> #41A681 </b># verde1</li>
    <li style='color:#3CA67F'><b> #3CA67F </b># verde2</li>
    <li style='color:#7ACAAB'><b> #7ACAAB </b># verde claro</li>
    <li style='color:#78a1a1'><b> #78a1a1 </b># azul</li>
    <li style='color:#8ABFB0'><b> #8ABFB0 </b># azul claro</li>
    <li style='color:#FFB27C'><b> #FFB27C </b># cor de pele clara</li>
    <li style='color:#F29877'><b> #F29877 </b># cor de pele</li>
    <li style='color:#ed845e'><b> #ed845e </b># laranja claro1</li>
    <li style='color:#D96236'><b> #D96236 </b># laranja claro2</li>
    <li style='color:#D95323'><b> #D95323 </b># laranja 1</li>
    <li style='color:#D94B18'><b> #D94B18 </b># laranja 2</li>
    <li style='color:#D9C2AD'><b> #D9C2AD </b># bege</li>
    <li style='color:#A66C4B'><b> #A66C4B </b># marrom claro</li>
    <li style='color:#86471B'><b> #86471B </b># marrom1</li>
    <li style='color:#732C02'><b> #732C02 </b># marrom2</li>
    <li style='color:#592202'><b> #592202 </b># marrom escuro1</li>
    <li style='color:#3D1806'><b> #3D1806 </b># marrom escuro2</li>
    <li style='color:#0D0D0D'><b> #0D0D0D </b># preto</li>
</ul>



In [74]:
# 220 NaNs
NewTable['order'].value_counts(dropna=False)

Phyllodocida        2430
Eunicida            1496
Sabellida            753
Scolecida            698
Amphinomida          464
Spionida             436
Terebellida          387
NaN                  242
Sipuncula              9
Canalipalpata          6
Crassiclitellata       1
Aspidosiphonida        1
Name: order, dtype: int64

In [75]:
# determinando cores de acordo com a planilha (2020.10.01 - IB e MN - Cores visualização.xlsx)
ordens = NewTable['order'].unique()
familias = NewTable['family'].unique()

# # o agrupamento é feito por famílias (ordem daquelas famílias deve assumir certa cor)
# cores_ordem = {
#     'Spionida':'#41A681',   # verde
#     'Sabellida':'#7ACAAB',  # verde claro
#     'Canalipalpata':'#78a1a1',  # azul
#     'Amphinomida':'#8ABFB0',  # azul claro
#     'Eunicida':'#A66C4B', # marrom claro
#     'Phyllodocida':'#732C02', # marrom2
#     'Terebellida':'#ed845e', # laranja claro1
#     'Scolecida':'#D94B18', # laranja 2
#     np.NAN:'#0D0D0D',  # preto
    
#     # ordens não citadas na planilha:
#     'Sipuncula':'#D9C2AD', # bege
#     'Crassiclitellata':'#FFB27C', # cor de pele clara
#     'Aspidosiphonida':'#F29877',  # cor de pele
    
# }

# paleta de cores por família
cores_familia = {
    'Magelonidae':'#238762',    # verde escuro 
    'Oweniidae':'#3CA67F',      # verde (centroide)  
    'Chaetopteridae':'#77c8a5', # verde
    'Amphinomidae':'#bbebd3',   # verde claro
    'Lumbrineridae':'#e7e5df',  # azul claro 1
    'Dorvilleidae':'#b2c0d0',   # azul claro2
    'Oenonidae':'#7A9FBF',      # azul (centroide)
    'Eunicidae':'#3c81ae',      # azul
    'Onuphidae':'#00669a',      # azul escuro
    'Syllidae':'#ffbd84', 
    'Typhloscolecidae':'#ffaa74', 
    'Aphroditidae':'#ff9760', 
    'Acoetidae':'#ff814b', 
    'Chrysopetalidae':'#fc6b36', 
    'Eulepethidae':'#eb5824',
    'Lopadorrhynchidae':'#d94814',  # laranja (centroide)
    'Polynoidae':'#c83b03',
    'Nereididae':'#b73000',
    'Nephtyidae':'#f18e56',
    'Glyceridae':'#D96236',         # laranja 2 (centroide)
    'Tomopteridae':'#bf381b',
    'Serpulidae':'#fbd0ad',
    'Sabellidae':'#f2b999', # cor de pele (centroide)
    'Sabellariidae':'#e8a287',
    'Spionidae':'#d27666',
    'Ampharetidae':'#a66c4b',  # marrom 1 (centroide)
    'Pectinariidae':'#975b39',
    'Trichobranchidae':'#874c2c',
    'Terebellidae':'#774124',
    'Cirratulidae':'#683720',
    'Flabelligeridae':'#eebd93',
    'Sternaspidae':'#dfa47a',
    'Orbiniidae':'#d28d60',
    'Capitellidae':'#c37746',
    'Arenicolidae':'#b4622f',
    'Cossuridae':'#a3501d',
    'Scalibregmatidae':'#92420e',
    'Paraonidae':'#823606',
    'Maldanidae':'#732c02', # marrom 2 (centroide)
    np.NAN:'#0D0D0D',  # preto
}

<br>


## Graphs

---

<br>

### Creating chart: counts per determiner per year

In [76]:
teste = NewTable.groupby(['determiner_full_name','cataloged_year']).count()['class'].reset_index().rename(columns=
                                                                                            {'class':'counts'})

In [78]:
# OBS: 2255 nomes faltando
NewTable['determiner_full_name'].isna().sum()

2255

In [80]:
# gráfico não tem todas as contagens por causa de nomes faltando
g1 = alt.Chart(teste, width=600, height=500).mark_circle().encode(
    x= alt.X('cataloged_year', type='ordinal', title='Cataloged Year'),
    y= alt.Y('determiner_full_name', type='nominal', title='Determiner Name', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts', scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    tooltip= alt.Tooltip(['determiner_full_name', 'cataloged_year', 'counts'])
)

# g1.save('./graphs/determiner/counts_per_determiner.html')

# g1

<font color='red' size='5'>mesmo gráfico, ordenando também pela ordem</font>

**OBS:** Note que

- as contagens mudam (porque estamos ordenando por um fator a mais)
- há pontos sobrepostos (semelhante ao que tinhamos para a base crustaceas - para um mesmo ano, um mesmo pesquisador descobriu animais de ordens/familias diferentes)

In [85]:
teste1 = NewTable.groupby(['determiner_full_name','cataloged_year', 'collection_prefix','family']).count()['class'].reset_index().rename(columns=
                                                                                            {'class':'counts'})

### Faceted chart version (needed some more flexibility, so I went for the hconcat version)

In [86]:
# g2 = alt.Chart(teste1, width=400, height=500).mark_circle().encode(
#     x= alt.X('cataloged_year', type='ordinal', title='Cataloged Year'),
#     y= alt.Y('determiner_full_name', type='nominal', title='Determiner Name', 
#             sort=alt.EncodingSortField('counts', op="count", order='descending')),
#     size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
#     color= alt.Color('order', type="nominal", title="Order", 
#                      scale=alt.Scale(domain=list(cores_ordem.keys()), range=list(cores_ordem.values()))),
#     tooltip= alt.Tooltip(['determiner_full_name', 'cataloged_year', 'counts', 'order'])
# )

# # saving chart
# # g2.facet(column='collection_prefix').resolve_scale(y='independent'
# #                                     ).save('./graphs/determiner/counts_per_determiner-w-order.html')

# g2.facet(column='collection_prefix').resolve_scale(y='independent', x='independent')

### hconcat: making each chart separately and concatenating them horizontally

In [93]:
g2 = alt.Chart(teste1[teste1['collection_prefix']=='IBUFRJ'], width=500, height=580,
              title='IBUFRJ').mark_circle().encode(
    x= alt.X('cataloged_year', type='ordinal', title='Cataloged Year'),
    y= alt.Y('determiner_full_name', type='nominal', title='Determiner Name', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 40)),
    tooltip= alt.Tooltip(['determiner_full_name', 'cataloged_year', 'counts', 'family'])
)

g3 = alt.Chart(teste1[teste1['collection_prefix']=='MNRJP'], width=100, height=580, 
              title='MNRJP').mark_circle().encode(
    x= alt.X('cataloged_year', type='ordinal', title='Cataloged Year'),
    y= alt.Y('determiner_full_name', type='nominal', title='Determiner Name', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts:Q', title='Counts', scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 40)),
    tooltip= alt.Tooltip(['determiner_full_name', 'cataloged_year', 'counts', 'family'])
)

graph = alt.hconcat(g2, g3)

# configuring graphic elements (font, fontSize, etc)
graph = graph.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# graph.save('./graphs/determiner/counts_per_determiner-w-family.html')

graph

alt.HConcatChart(...)

<br>

<font color='red'>**mesmo gráfico, agora ordenando pelo primeiro ano que o pesquisador aparece na base**</font>

In [96]:
# ordenando
temp2 = teste1[teste1['collection_prefix']=='IBUFRJ'].copy()
temp3 = teste1[teste1['collection_prefix']=='MNRJP'].copy()

temp2.sort_values(['cataloged_year', 'determiner_full_name'], inplace=True)
temp3.sort_values(['cataloged_year', 'determiner_full_name'], inplace=True)

# salvando ordem das entradas
sorting2 = list(temp2['determiner_full_name'].unique())
sorting3 = list(temp3['determiner_full_name'].unique())

In [100]:
g2 = alt.Chart(teste1[teste1['collection_prefix']=='IBUFRJ'], width=500, height=580,
              title='IBUFRJ').mark_circle().encode(
    x= alt.X('cataloged_year', type='ordinal', title='Cataloged Year'),
    y= alt.Y('determiner_full_name', type='nominal', title='Determiner Name', 
            sort=sorting2),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 40)),
    tooltip= alt.Tooltip(['determiner_full_name', 'cataloged_year', 'counts', 'family'])
)

g3 = alt.Chart(teste1[teste1['collection_prefix']=='MNRJP'], width=100, height=580, 
              title='MNRJP').mark_circle().encode(
    x= alt.X('cataloged_year', type='ordinal', title='Cataloged Year'),
    y= alt.Y('determiner_full_name', type='nominal', title='Determiner Name', 
            sort=sorting3),
    size= alt.Size('counts:Q', title='Counts', scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 40)),
    tooltip= alt.Tooltip(['determiner_full_name', 'cataloged_year', 'counts', 'family'])
)

graph = alt.hconcat(g2, g3)

# configuring graphic elements (font, fontSize, etc)
graph = graph.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# graph.save('./graphs/determiner/counts_per_determiner-w-family-reordenado.html')

graph

alt.HConcatChart(...)

<br>

### Gráfico dos determinadores mais expressivos 

<font color='red' size='5'>mesmo gráfico, ordenado pela soma das contagens </font>

In [103]:
# somando contagens de cada pesquisador
sorting2 = temp2.groupby('determiner_full_name').sum()['counts'].reset_index().rename(
    columns={'counts':'sum'})
sorting3 = temp3.groupby('determiner_full_name').sum()['counts'].reset_index().rename(
    columns={'counts':'sum'})

sorting2 = sorting2.sort_values('sum', ascending=False)
sorting3 = sorting3.sort_values('sum', ascending=False)

# Nomes dos pesquisadores ordenados
sort_list2 = sorting2['determiner_full_name'].unique()
sort_list3 = sorting3['determiner_full_name'].unique()

In [105]:
g2 = alt.Chart(teste1[teste1['collection_prefix']=='IBUFRJ'], width=500, height=580,
              title='IBUFRJ').mark_circle().encode(
    x= alt.X('cataloged_year', type='ordinal', title='Cataloged Year'),
    y= alt.Y('determiner_full_name', type='nominal', title='Determiner Name', 
            sort=sort_list2),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 40)),
    tooltip= alt.Tooltip(['determiner_full_name', 'cataloged_year', 'counts', 'family'])
)

g3 = alt.Chart(teste1[teste1['collection_prefix']=='MNRJP'], width=100, height=580, 
              title='MNRJP').mark_circle().encode(
    x= alt.X('cataloged_year', type='ordinal', title='Cataloged Year'),
    y= alt.Y('determiner_full_name', type='nominal', title='Determiner Name', 
            sort=sort_list3),
    size= alt.Size('counts:Q', title='Counts', scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 40)),
    tooltip= alt.Tooltip(['determiner_full_name', 'cataloged_year', 'counts', 'family'])
)

graph = alt.hconcat(g2, g3)

# configuring graphic elements (font, fontSize, etc)
graph = graph.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# graph.save('./graphs/determiner/counts_per_determiner-w-family-ordenado_pela_soma.html')

graph

alt.HConcatChart(...)

<br>

### Creating chart: counts per collector per year

<font color='red' size='5'> Collectors name is sensitive data. Do not publish it without curator's permission </font>

In [119]:
# OBS: 1059 NANs
NewTable['collector_full_name'].isna().sum()

1439

In [120]:
teste = NewTable.groupby(['collector_full_name','start_year']).count()['class'].reset_index().rename(columns=
                                                                                            {'class':'counts'})

In [124]:
g1 = alt.Chart(teste1[teste1['collection_prefix']=='IBUFRJ'], width=500, height=800,
              title='IBUFRJ').mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('collector_full_name', type='nominal', title='Collector Name', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts', scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    tooltip= alt.Tooltip(['collector_full_name', 'start_year', 'counts'])
)

g2 = alt.Chart(teste1[teste1['collection_prefix']=='MNRJP'], width=500, height=800,
              title='MNRJP').mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('collector_full_name', type='nominal', title='Collector Name', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts', scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    tooltip= alt.Tooltip(['collector_full_name', 'start_year', 'counts'])
)

graph = alt.hconcat(g1, g2)

# configuring graphic elements (font, fontSize, etc)
graph = graph.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# graph.save('./graphs/collector/counts_per_collector.html')

# graph

<font color='red' size='5'>mesmo gráfico, colorindo por família</font>

**OBS:** Note que

- as contagens mudam (porque estamos ordenando por um fator a mais)
- há pontos sobrepostos (semelhante ao que tinhamos para a base crustaceas - para um mesmo ano, um mesmo pesquisador descobriu animais de ordens/familias diferentes)

In [125]:
teste1 = NewTable.groupby(['collector_full_name','start_year', 'family', 'collection_prefix']).count()['class'].reset_index().rename(columns=
                                                                                            {'class':'counts'})

In [127]:
g2 = alt.Chart(teste1[teste1['collection_prefix']=='IBUFRJ'], width=500, height=800,
              title='IBUFRJ').mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('collector_full_name', type='nominal', title='Collector Name', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 40)),
    tooltip= alt.Tooltip(['collector_full_name', 'start_year', 'counts', 'family'])
)

g3 = alt.Chart(teste1[teste1['collection_prefix']=='MNRJP'], width=500, height=800,
              title='MNRJP').mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('collector_full_name', type='nominal', title='Collector Name', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 40)),
    tooltip= alt.Tooltip(['collector_full_name', 'start_year', 'counts', 'family'])
)

graph = alt.hconcat(g2, g3)

# configuring graphic elements (font, fontSize, etc)
graph = graph.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# graph.save('./graphs/collector/counts_per_collector-w-family.html')

# graph

<br>

<font color='red'>**mesmo gráfico, agora ordenando pelo primeiro ano que o pesquisador aparece na base**</font>

In [25]:
# ordenando
teste1.sort_values(['start_year', 'collector_full_name'], inplace=True)

# salvando ordem das entradas
sorting = list(teste1['collector_full_name'].unique())

In [130]:
g2 = alt.Chart(teste1[teste1['collection_prefix']=='IBUFRJ'], width=500, height=800,
              title='IBUFRJ').mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('collector_full_name', type='nominal', title='Collector Name', 
            sort=alt.EncodingSortField('start_year', op="min", order='ascending')),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 40)),
    tooltip= alt.Tooltip(['collector_full_name', 'start_year', 'counts', 'family'])
)

g3 = alt.Chart(teste1[teste1['collection_prefix']=='MNRJP'], width=500, height=800,
              title='MNRJP').mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('collector_full_name', type='nominal', title='Collector Name', 
            sort=alt.EncodingSortField('start_year', op="min", order='ascending')),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 40)),
    tooltip= alt.Tooltip(['collector_full_name', 'start_year', 'counts', 'family'])
)

graph = alt.hconcat(g2, g3)

# configuring graphic elements (font, fontSize, etc)
graph = graph.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# graph.save('./graphs/collector/counts_per_collector-w-family-reordenado.html')

# graph

<br>

### Gráfico dos coletores mais expressivos

In [135]:
temp1.head()

,collector_full_name,start_year,family,collection_prefix,counts
0,Alexandre Villas Boas,2005.0,Chrysopetalidae,IBUFRJ,2
1,Alexandre Villas Boas,2005.0,Dorvilleidae,IBUFRJ,1
2,Alexandre Villas Boas,2005.0,Eunicidae,IBUFRJ,1
3,Alexandre Villas Boas,2005.0,Oenonidae,IBUFRJ,2
4,Alexandre Villas Boas,2005.0,Polynoidae,IBUFRJ,4


In [131]:
temp1 = teste1[teste1['collection_prefix']=='IBUFRJ']
temp2 = teste1[teste1['collection_prefix']=='MNRJP']

# somando contagens de cada pesquisador
sorting1 = temp1.groupby('collector_full_name').sum()['counts'].reset_index().rename(
    columns={'counts':'sum'})
sorting2 = temp2.groupby('collector_full_name').sum()['counts'].reset_index().rename(
    columns={'counts':'sum'})

sorting1 = sorting1.sort_values('sum', ascending=False)
sorting2 = sorting2.sort_values('sum', ascending=False)

# Nomes dos pesquisadores ordenados
sort_list1 = sorting1['collector_full_name'].unique()
sort_list2 = sorting2['collector_full_name'].unique()

# sorting1.head()

In [138]:
g2 = alt.Chart(temp1[temp1['collector_full_name'].isin(sort_list1)], width=500, height=800,
              title='IBUFRJ').mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('collector_full_name', type='nominal', title='Collector Name', 
            sort=sort_list1[:50]),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 40)),
    tooltip= alt.Tooltip(['collector_full_name', 'start_year', 'counts', 'family'])
)

g3 = alt.Chart(temp2[temp2['collector_full_name'].isin(sort_list2)], width=500, height=800,
              title='MNRJP').mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('collector_full_name', type='nominal', title='Collector Name', 
            sort=sort_list2[:50]),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 40)),
    tooltip= alt.Tooltip(['collector_full_name', 'start_year', 'counts', 'family'])
)

graph = alt.hconcat(g2, g3)

# configuring graphic elements (font, fontSize, etc)
graph = graph.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# graph.save('./graphs/collector/counts_per_collector-ordenado_pela_soma.html')

graph

alt.HConcatChart(...)

<br>

**The end!**

-----

<font color='red' size='5'>**Experimental:** fazendo com primeiro e segundo nome dos coletores </font>

problema: temos poucos nomes para o segundo coletor

In [139]:
# juntando coletor 1 e coletor 2
NewTable['collectors'] = NewTable['collector_full_name'].astype(str) + ', ' + NewTable['collector_full_name2'].astype(str)

In [140]:
teste1 = NewTable.groupby(['collectors','start_year', 'family', 'collection_prefix']).count()['class'].reset_index().rename(columns=
                                                                                            {'class':'counts'})

In [145]:
g2 = alt.Chart(teste1, width=600, height=900).mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('collectors', type='nominal', title='Collector 1, Collector 2', 
            sort=alt.EncodingSortField('counts', op="sum", order='descending')),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit=40)),
    tooltip= alt.Tooltip(['collectors', 'start_year', 'counts', 'family'])
)

# faceting
g2 = g2.facet(column='collection_prefix').resolve_scale(y='independent')

# saving chart
# g2.save('./graphs/collector/counts_per_collector-w-order-coletor1e2.html')

g2

alt.FacetChart(...)

<br>

<font color='red'>**mesmo gráfico, agora ordenando pelo primeiro ano que o pesquisador aparece na base**</font>

In [146]:
# ordenando
teste1.sort_values(['start_year', 'collectors'], inplace=True)

# salvando ordem das entradas
sorting = list(teste1['collectors'].unique())

In [147]:
g2 = alt.Chart(teste1, width=600, height=900).mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('collectors', type='nominal', title='Collector 1, Collector 2', 
            sort=alt.EncodingSortField('start_year', op="min", order='ascending')),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('family:N', title="Family", 
                     scale=alt.Scale(domain=list(cores_familia.keys()), range=list(cores_familia.values())),
                     legend= alt.Legend(columns=2, symbolLimit=40)),
    tooltip= alt.Tooltip(['collectors', 'start_year', 'counts', 'family'])
)

g2 = g2.facet(column='collection_prefix').resolve_scale(y='independent')

# saving chart
# g2.save('./graphs/collector/counts_per_researcher-c-ordem_reordenado-coletor1e2.html')

g2

alt.FacetChart(...)

<br>

**That's it!**